In [ ]:
# Training parameters

window = 672       # 1 week
stride = 4         # 1 hour
latent_dim = 6     # Autoencoder latent dimension
epochs = 150       # Number of epochs
batch_size = 8     # Batch size

In [ ]:
# Building the model

input_shape = X_train.shape[1:]
output_shape = X_train.shape[1:]

#Encoder
encoder_input = tf.keras.Input(shape=input_shape)

convlstm = tfkl.LSTM(64, return_sequences=True)(encoder_input)
convlstm = tfkl.LSTM(latent_dim, return_sequences=False)(convlstm)

#Decoder
convlstm = tfkl.RepeatVector(window)(convlstm)
convlstm = tfkl.LSTM(64, return_sequences=True)(convlstm)
decoder_output = tfkl.TimeDistributed(tfkl.Dense(output_shape[1]))(convlstm)

#Model
autoencoder = tfk.Model(inputs=encoder_input, outputs=decoder_output, name='Autoencoder')
autoencoder.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=[tfk.metrics.MeanSquaredError()])
autoencoder.summary()

In [ ]:
# Training
history = autoencoder.fit(
    X_train,
    X_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = (X_val,X_val),
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
).history